In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head(5)

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [2]:
df.isnull().sum().sum()

null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()


lang             4
retweet_count    4
dtype: int64

In [3]:
df.lang.unique()
df.lang.nunique()

232

In [4]:
print(df.loc[0]['original_text'])

Happy #MothersDay to all you amazing mothers out there! I know it's hard not being able to see your mothers today but it's on all of us to do what we can to protect the most vulnerable members of our society. #BeatCoronaVirus pic.twitter.com/va4nFjFQ5B


In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
#docs = np.array([df.loc[0]['original_text'],df.loc[1]['original_text'],df.loc[2]['original_text']])
bag = count.fit_transform(df.loc[:]['original_text'])

In [6]:
print(count.vocabulary_)

{'happy': 6698, 'mothersday': 9275, 'to': 13166, 'all': 1539, 'you': 14509, 'amazing': 1591, 'mothers': 9266, 'out': 10172, 'there': 13026, 'know': 7983, 'it': 7525, 'hard': 6726, 'not': 9838, 'being': 2224, 'able': 1277, 'see': 11809, 'your': 14520, 'today': 13172, 'but': 2737, 'on': 10057, 'of': 9973, 'us': 13601, 'do': 4858, 'what': 14040, 'we': 13950, 'can': 2940, 'protect': 10832, 'the': 12976, 'most': 9226, 'vulnerable': 13842, 'members': 8941, 'our': 10166, 'society': 12217, 'beatcoronavirus': 2164, 'pic': 10488, 'twitter': 13382, 'com': 3626, 'va4nfjfq5b': 13658, 'day': 4439, 'mum': 9372, 'sorry': 12272, 'be': 2145, 'bring': 2625, 'flowers': 5918, 'cwtch': 4231, 'honestly': 7003, 'at': 1889, 'this': 13066, 'point': 10624, 'walk': 13882, 'hot': 7038, 'coals': 3584, 'll': 8360, 'with': 14155, 'bells': 2243, 'as': 1843, 'soon': 12253, 'love': 8455, 'lots': 8443, 'xxx': 14395, 'need': 9654, 'more': 9215, 'photos': 10481, 'https': 7090, 'app': 1754, 'goo': 6414, 'gl': 6338, 'm3vxblr

In [7]:
print(bag.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
np.set_printoptions(precision=2)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(df.loc[:]['original_text'])).toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


In [11]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(df.loc[:]['original_text'])).toarray()[-1]
raw_tfidf

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [14]:
df['original_text'] = df['original_text'].apply(preprocessor)

In [15]:
df.loc[0]['original_text']

'happy mothersday to all you amazing mothers out there i know it s hard not being able to see your mothers today but it s on all of us to do what we can to protect the most vulnerable members of our society beatcoronavirus pic twitter com va4nfjfq5b'

In [16]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [17]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nitaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter(df.loc[0]['original_text'])
if w not in stop]

['happi',
 'mothersday',
 'amaz',
 'mother',
 'know',
 'hard',
 'abl',
 'see',
 'mother',
 'today',
 'us',
 'protect',
 'vulner',
 'member',
 'societi',
 'beatcoronaviru',
 'pic',
 'twitter',
 'com',
 'va4nfjfq5b']

In [19]:
X_train = df.loc[:2500, 'original_text'].values
y_train = df.loc[:2500, 'sentiment_class'].values
X_test = df.loc[2501:, 'original_text'].values
y_test = df.loc[2501:, 'sentiment_class'].values
print(X_test)

['happy mother s day mothersday cava rosecupcakes marksandspencer sundayfizztime https www instagram com p b djph8h3clh jzvray1ehlmu67q19xdjqxvxcg0 igshid 14efil4rhltut '
 ' heads_together posted this message and we thought it s so therapeutic and appropriate for this moment mothersday can be a day of mixed emotions for many people for some it can be a happy day of celebration https www instagram com p b ccfcnj2qo igshid 1ixbhztdv0nag '
 'meanwhile i know y all didn t notice but happy mother s day our mothers are real superwomen s t a y s a f e lifeofpatetm happymothersday mothersday nigerianmothers quarantine covid_19 https www instagram com p b cwhdojyzz igshid jys8b1nmvv77 '
 'not impressed at all waitrose just spoken to someone on customer services regarding my flower order for my mum for mothers day that didnt arrive just to be told somethings gone majorly wrong and did i want to reorder no apology or other options given not happy '
 'i m in bed with fierce body aches and a burnin

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                       tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

X = tfidf.fit_transform(X_train)
X2 = tfidf.transform(X_test)
X.shape
X2.shape
clf = LogisticRegressionCV(cv =5,
                          scoring='recall',
                          random_state=0,
                          n_jobs=1,
                          verbose=3,
                          max_iter=300).fit(X,y_train)






C:\Users\Nitaya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   28.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.5min finished


In [21]:
#print(X)
#print(X2)
y = clf.predict(X2)
print(y)
print(y_test)

[ 0  0  0  0  1  0 -1  0  0  0  1  1  0  0  0  1  1  1  0  0  0  1 -1  0
  1  0 -1  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0
  0  1 -1  0  1  0 -1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 -1 -1  0 -1  0  1  0  0 -1  0  0  1  0  0  0  0  0  0  0  0  0  0
  1  0  0  0  0  0  1  0 -1  0  0  0  0 -1  0  1  0  0  1 -1  0  0  0  0
  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  1 -1  0  1 -1  1  0  1
  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  1  0  0  0  0
  0  1  1  0  0  0  0  0  0  0 -1  1  1  0  0  0  0  0 -1  1  0  0  0  0
  1  1 -1  1  0  1 -1  1  0  0 -1  0  0  0 -1  0 -1  1  0  0 -1  0  0  0
  0  0  0  0  1  0  0  0  0  1  0  0  1  0  0  0 -1 -1  0  1  0  0 -1  0
  0  0  0  0  0 -1  0  0  0  1  0 -1  0  0  0  0  1  0  0  0  0  0  0 -1
  0  0  0  0  0  0 -1  0  0  1  0  0  0  0 -1 -1  0  1  0  1  1  0 -1  1
  0  0  1  0  0  0  1  0  0  0  1  0  0  0 -1  0  0  1  0  0  0  0 -1  0
  0  0  0  0  0 -1  0  0  0  0  0  0  0  0 -1  0  0

In [22]:
from sklearn.metrics import accuracy_score,f1_score
value = accuracy_score(y_test,y)
print(value)
score = 100*f1_score(y_test,y,average='weighted')
print(score)

0.4373297002724796
40.2988793878346


In [23]:
test_data =pd.read_csv('test.csv')
test_data = test_data.original_text.values
X3 = tfidf.transform(test_data)
y3 = clf.predict(X3)
print(y3)

[0 0 0 ... 0 1 1]


In [24]:
test_data =pd.read_csv('test.csv')
final = pd.DataFrame()
final['id'] = test_data.loc[:]['id']
print(final)

                id
0     1.246628e+18
1     1.245898e+18
2     1.244717e+18
3     1.245730e+18
4     1.244636e+18
5     1.244048e+18
6     1.243919e+18
7     1.246245e+18
8     1.246139e+18
9     1.244255e+18
10    1.244233e+18
11    1.244249e+18
12    1.245385e+18
13    1.245410e+18
14    1.245083e+18
15    1.246114e+18
16    1.244642e+18
17    1.245881e+18
18    1.246143e+18
19    1.244614e+18
20    1.245671e+18
21    1.244664e+18
22    1.243922e+18
23    1.244361e+18
24    1.246459e+18
25    1.244256e+18
26    1.243953e+18
27    1.245803e+18
28    1.245058e+18
29    1.245847e+18
...            ...
1357  1.246438e+18
1358  1.245348e+18
1359  1.246629e+18
1360  1.244266e+18
1361  1.244698e+18
1362  1.245423e+18
1363  1.243917e+18
1364  1.243977e+18
1365  1.243966e+18
1366  1.244579e+18
1367  1.243961e+18
1368  1.245644e+18
1369  1.246404e+18
1370  1.245822e+18
1371  1.245040e+18
1372  1.245367e+18
1373  1.243959e+18
1374  1.245044e+18
1375  1.245755e+18
1376  1.245976e+18
1377  1.2464

In [25]:
final['sentiment_class'] = y3[:]

In [26]:
print(final)

                id  sentiment_class
0     1.246628e+18                0
1     1.245898e+18                0
2     1.244717e+18                0
3     1.245730e+18                1
4     1.244636e+18                0
5     1.244048e+18                0
6     1.243919e+18                1
7     1.246245e+18                1
8     1.246139e+18                0
9     1.244255e+18                1
10    1.244233e+18                0
11    1.244249e+18                0
12    1.245385e+18                0
13    1.245410e+18                0
14    1.245083e+18                0
15    1.246114e+18                0
16    1.244642e+18                1
17    1.245881e+18                0
18    1.246143e+18                0
19    1.244614e+18                0
20    1.245671e+18                0
21    1.244664e+18                0
22    1.243922e+18               -1
23    1.244361e+18                0
24    1.246459e+18                0
25    1.244256e+18                0
26    1.243953e+18          

In [27]:
final.to_csv(r'C:\Users\Nitaya\Desktop\sem6\Machine Learning\Hacker Earth - Tweets Sentiment\dataset\final2.csv',sep=',',encoding='utf-8',index=False)